In [1]:
import pandas as pd
import numpy as np
import urllib3
import matplotlib.pyplot as plt
import requests
%matplotlib inline

In [3]:
request = urllib3.connection_from_url('https://municipaldata.treasury.gov.za/api/cubes/')

In [28]:
cubes = requests.get('https://municipaldata.treasury.gov.za/api/cubes/', allow_redirects=True).json()

In [29]:
data = pd.DataFrame(cubes['data'])

In [36]:
aged = requests.get(data.uri[0])

In [44]:
data[

'https://municipaldata.treasury.gov.za/api/cubes//uifwexp'

In [21]:
for row in r.json()['data']:
    print(row['name'])

aged_creditor
conditional_grants
incexp
aged_debtor
capital
uifwexp
cflow
municipalities
audit_opinions
officials
demarcation_changes
repmaint
bsheet
